# Scratch JSON-formatted Fund Details

In [1]:
import requests
import json
import re
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
urlForFundDetail = "http://api.fund.eastmoney.com/f10/lsjz?fundCode=161725&pageIndex=1&pageSize=9999&startDate=&endDate="
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36', 'Referer': 'http://fundf10.eastmoney.com/'}
httpRequest = requests.get(urlForFundDetail, headers = headers)

In [3]:
httpRequest?

Type:        Response
String form: <Response [200]>
File:        ~/.local/lib/python3.6/site-packages/requests/models.py
Docstring:  
The :class:`Response <Response>` object, which contains a
server's response to an HTTP request.


In [4]:
rawJsonString = httpRequest.content.decode('utf-8')
rawJsonString?

Type:        str
String form: {"Data":{"LSJZList":[{"LJJZ":"2.9776","JZZZL":"-1.67","SGZT":"开放申购","SHZT":"开放赎回","FHFCZ":"","FHF <...> 054"},"ErrCode":0,"ErrMsg":null,"TotalCount":1449,"Expansion":null,"PageSize":9999,"PageIndex":1}
Length:      269946
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.


# Extract JSON Array of Time Series from JSON String

In [5]:
tidyJsonString = re.sub(r'^.*?([{].+[}]).*$', r'\1', rawJsonString)

In [6]:
tidyJsonString?

Type:        str
String form: {"Data":{"LSJZList":[{"LJJZ":"2.9776","JZZZL":"-1.67","SGZT":"开放申购","SHZT":"开放赎回","FHFCZ":"","FHF <...> 054"},"ErrCode":0,"ErrMsg":null,"TotalCount":1449,"Expansion":null,"PageSize":9999,"PageIndex":1}
Length:      269946
Docstring:  
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.


# Parse JSON String into pandas.DataFrame

In [7]:
jsonObject = json.loads(tidyJsonString)
jsonObject?

Type:        dict
String form: {'Data': {'LSJZList': [{'LJJZ': '2.9776', 'JZZZL': '-1.67', 'SGZT': '开放申购', 'SHZT': '开放赎回', 'FHFC <...> ode': 0, 'ErrMsg': None, 'TotalCount': 1449, 'Expansion': None, 'PageSize': 9999, 'PageIndex': 1}
Length:      7
Docstring:  
dict() -> new empty dictionary
dict(mapping) -> new dictionary initialized from a mapping object's
    (key, value) pairs
dict(iterable) -> new dictionary initialized as if via:
    d = {}
    for k, v in iterable:
        d[k] = v
dict(**kwargs) -> new dictionary initialized with the name=value pairs
    in the keyword argument list.  For example:  dict(one=1, two=2)


In [8]:
(jsonObject.keys(), len(jsonObject['Data']['LSJZList']), jsonObject['TotalCount'])

(dict_keys(['Data', 'ErrCode', 'ErrMsg', 'TotalCount', 'Expansion', 'PageSize', 'PageIndex']),
 1449,
 1449)

In [9]:
df = pd.DataFrame(jsonObject['Data']['LSJZList'])
set(df.columns.to_list())

{'ACTUALSYI',
 'DTYPE',
 'DWJZ',
 'FHFCBZ',
 'FHFCZ',
 'FHSP',
 'FSRQ',
 'JZZZL',
 'LJJZ',
 'NAVTYPE',
 'SDATE',
 'SGZT',
 'SHZT'}

# Drop Unnecessary Columns & Fix the Index

In [10]:
columns_to_keep = {'FSRQ', 'DWJZ', 'LJJZ', 'JZZZL'}
columns_to_drop = [ key for key in (set(df.columns.to_list()) -  columns_to_keep) ]
df = df.drop(columns = columns_to_drop)

df.index = df.FSRQ
# convert data type from string to numeric
for floatColumn in ['DWJZ', 'LJJZ', 'JZZZL']:
    df[floatColumn] = pd.to_numeric(df[floatColumn], errors = 'coerce')

df

,LJJZ,JZZZL,FSRQ,DWJZ
FSRQ,,,,
2021-05-07,2.9776,-1.67,2021-05-07,1.3465
2021-05-06,3.0005,-3.05,2021-05-06,1.3694
2021-04-30,3.0436,-1.16,2021-04-30,1.4125
2021-04-29,3.0602,-0.12,2021-04-29,1.4291
2021-04-28,3.0619,2.51,2021-04-28,1.4308
...,...,...,...,...
2015-06-08,1.0380,2.57,2015-06-08,1.0380
2015-06-05,1.0120,1.50,2015-06-05,1.0120
2015-06-04,0.9970,NaN,2015-06-04,0.9970


# Fix the Missing Values in Column "JZZZL"

In [11]:
def woody_neighbor_rows(row_index, df, rowCount = 1):
    df_index = df.index
    row_index_loc = df_index.get_loc(row_index)
    return df.loc[df_index[(row_index_loc):(row_index_loc + rowCount)]]

def woody_next_row(row_index, df):
    available_df = woody_neighbor_rows(row_index, df, 2)
    return woody_neighbor_rows(row_index, df, 2).iloc[1] if 2 == available_df.index.size else None

# Test
print(woody_neighbor_rows('2015-06-29', df, 5))
print(woody_neighbor_rows('2015-06-29', df, 1))
print(woody_neighbor_rows('2015-05-27', df, 5))

# Test
woody_next_row('2015-05-29', df)

def fix_nan_in_jzzzl(row, df):
    current_day_data = row
    previous_day_data = woody_next_row(row.name, df) if woody_next_row(row.name, df) is not None else row
    row['JZZZL'] = (current_day_data['LJJZ'] - previous_day_data['LJJZ']) / previous_day_data['DWJZ'] * 100
    # print(row.name, "row['JZZZL']", row['JZZZL'], "\n", "current_day_data['LJJZ']", current_day_data['LJJZ'], "\n", "previous_day_data['LJJZ']", previous_day_data['LJJZ'], "\n\n")
    return row

nan_jzzzl_df = df[df['JZZZL'].isna()]
nan_jzzzl_df = nan_jzzzl_df.apply(lambda row: fix_nan_in_jzzzl(row, df), axis = 1)
df.update(nan_jzzzl_df)
df

             LJJZ  JZZZL        FSRQ   DWJZ
FSRQ                                       
2015-06-29  1.023   2.51  2015-06-29  1.023
2015-06-26  0.998  -5.94  2015-06-26  0.998
2015-06-25  1.061  -2.03  2015-06-25  1.061
2015-06-24  1.083   6.07  2015-06-24  1.083
2015-06-23  1.021   6.02  2015-06-23  1.021
             LJJZ  JZZZL        FSRQ   DWJZ
FSRQ                                       
2015-06-29  1.023   2.51  2015-06-29  1.023
            LJJZ  JZZZL        FSRQ  DWJZ
FSRQ                                     
2015-05-27   1.0    NaN  2015-05-27   1.0


,LJJZ,JZZZL,FSRQ,DWJZ
FSRQ,,,,
2021-05-07,2.9776,-1.670000,2021-05-07,1.3465
2021-05-06,3.0005,-3.050000,2021-05-06,1.3694
2021-04-30,3.0436,-1.160000,2021-04-30,1.4125
2021-04-29,3.0602,-0.120000,2021-04-29,1.4291
2021-04-28,3.0619,2.510000,2021-04-28,1.4308
...,...,...,...,...
2015-06-08,1.0380,2.570000,2015-06-08,1.0380
2015-06-05,1.0120,1.500000,2015-06-05,1.0120
2015-06-04,0.9970,0.201005,2015-06-04,0.9970


# Calculate Ideal DWJZ

In [12]:
df['IDEAL_DWJZ'] = 1.0
ideal_dwjz_series = df['IDEAL_DWJZ']

def make_ideal_dwjz(row, df):
    row_index = row['FSRQ']
    next_row = woody_next_row(row_index, df)
    if next_row is not None:
        next_row_index = next_row['FSRQ']
        change_rate_of_next_trade_day = next_row['JZZZL'] 
        next_ideal_dwjz = ideal_dwjz_series[row_index] * (100.0 + change_rate_of_next_trade_day) / 100.0
        ideal_dwjz_series.at[next_row_index] = next_ideal_dwjz
    else:
        pass
    return row
reversed_df = df.sort_index()
reversed_df['IDEAL_DWJZ'] = 1.0
reversed_df = reversed_df.apply(lambda row: make_ideal_dwjz(row, reversed_df), axis = 1)

df.update(ideal_dwjz_series)
df

,LJJZ,JZZZL,FSRQ,DWJZ,IDEAL_DWJZ
FSRQ,,,,,
2021-05-07,2.9776,-1.670000,2021-05-07,1.3465,5.109489
2021-05-06,3.0005,-3.050000,2021-05-06,1.3694,5.196267
2021-04-30,3.0436,-1.160000,2021-04-30,1.4125,5.359739
2021-04-29,3.0602,-0.120000,2021-04-29,1.4291,5.422641
2021-04-28,3.0619,2.510000,2021-04-28,1.4308,5.429156
...,...,...,...,...,...
2015-06-08,1.0380,2.570000,2015-06-08,1.0380,1.037962
2015-06-05,1.0120,1.500000,2015-06-05,1.0120,1.011955
2015-06-04,0.9970,0.201005,2015-06-04,0.9970,0.997000


# Calculate Periodical Aggregative Metrics

In [13]:
df[['LJJZ_2W_MAX', 'LJJZ_2W_MIN', 'LJJZ_4W_MAX', 'LJJZ_4W_MIN', 'LJJZ_6W_MAX', 'LJJZ_6W_MIN', 'IDEAL_DWJZ_4W_MAX', 'IDEAL_DWJZ_4W_MIN']] = 0.0

def n_week_max(n_week, row_index, df):
    n_week_df = woody_neighbor_rows(row_index, df, n_week * 5)
    return n_week_df['LJJZ'].max()

def n_week_min(n_week, row_index, df):
    n_week_df = woody_neighbor_rows(row_index, df, n_week * 5)
    return n_week_df['LJJZ'].min()

def make_min_max_of_periodical_ljjz(row, df):
    row_index = str(row['FSRQ'])
    
    n_week_df = woody_neighbor_rows(row_index, df, 2 * 5)
    df.at[row['FSRQ'], 'LJJZ_2W_MAX'] = n_week_df['LJJZ'].max()
    df.at[row['FSRQ'], 'LJJZ_2W_MIN'] = n_week_df['LJJZ'].min()
    df.at[row['FSRQ'], 'LJJZ_2W_INC'] = n_week_df['LJJZ'].max()
    df.at[row['FSRQ'], 'LJJZ_2W_DEC'] = n_week_df['LJJZ'].min()
    n_week_df = woody_neighbor_rows(row_index, df, 4 * 5)
    df.at[row['FSRQ'], 'LJJZ_4W_MAX'] = n_week_df['LJJZ'].max()
    df.at[row['FSRQ'], 'LJJZ_4W_MIN'] = n_week_df['LJJZ'].min()
    n_week_df = woody_neighbor_rows(row_index, df, 6 * 5)
    df.at[row['FSRQ'], 'LJJZ_6W_MAX'] = n_week_df['LJJZ'].max()
    df.at[row['FSRQ'], 'LJJZ_6W_MIN'] = n_week_df['LJJZ'].min()
    
    period_by_week = 4
    
    n_week_df = woody_neighbor_rows(row_index, df, period_by_week * 5)
    ideal_dwjz_4w_max = n_week_df['IDEAL_DWJZ'].max()
    ideal_dwjz_4w_min = n_week_df['IDEAL_DWJZ'].min()
    df.at[row_index, 'IDEAL_DWJZ_4W_MAX'] = ideal_dwjz_4w_max
    df.at[row_index, 'IDEAL_DWJZ_4W_MIN'] = ideal_dwjz_4w_min

    row

df.apply(lambda row: make_min_max_of_periodical_ljjz(row, df), axis = 1)

df

,LJJZ,JZZZL,FSRQ,DWJZ,IDEAL_DWJZ,LJJZ_2W_MAX,LJJZ_2W_MIN,LJJZ_4W_MAX,LJJZ_4W_MIN,LJJZ_6W_MAX,LJJZ_6W_MIN,IDEAL_DWJZ_4W_MAX,IDEAL_DWJZ_4W_MIN,LJJZ_2W_INC,LJJZ_2W_DEC
FSRQ,,,,,,,,,,,,,,,
2021-05-07,2.9776,-1.670000,2021-05-07,1.3465,5.109489,3.0619,2.9776,3.0619,2.8972,3.0619,2.8456,5.429156,4.804428,3.0619,2.9776
2021-05-06,3.0005,-3.050000,2021-05-06,1.3694,5.196267,3.0619,2.9794,3.0619,2.8972,3.0619,2.8424,5.429156,4.804428,3.0619,2.9794
2021-04-30,3.0436,-1.160000,2021-04-30,1.4125,5.359739,3.0619,2.9642,3.0619,2.8972,3.0619,2.8369,5.429156,4.804428,3.0619,2.9642
2021-04-29,3.0602,-0.120000,2021-04-29,1.4291,5.422641,3.0619,2.9479,3.0619,2.8972,3.0619,2.8369,5.429156,4.804428,3.0619,2.9479
2021-04-28,3.0619,2.510000,2021-04-28,1.4308,5.429156,3.0619,2.9163,3.0619,2.8951,3.0619,2.8345,5.429156,4.796729,3.0619,2.9163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-06-08,1.0380,2.570000,2015-06-08,1.0380,1.037962,1.0380,0.9950,1.0380,0.9950,1.0380,0.9950,1.037962,0.995000,1.0380,0.9950
2015-06-05,1.0120,1.500000,2015-06-05,1.0120,1.011955,1.0120,0.9950,1.0120,0.9950,1.0120,0.9950,1.011955,0.995000,1.0120,0.9950
2015-06-04,0.9970,0.201005,2015-06-04,0.9970,0.997000,1.0000,0.9950,1.0000,0.9950,1.0000,0.9950,1.000000,0.995000,1.0000,0.9950


# Calculate Inc/Dec Speed in One Month

Calculation Strategy
* Period is 4 weeks
* Find "max/min" IDEAL_DWJZ
* Take each day's IDEAL_DWJZ and above "max/min" to calculate increasement/decreasement rate in the period, which is noted as "inc/dec" amount
* Find the days' interval between each day and its corresponding "max/min" day, which is noted as days
* Calculate "inc/dec" speed using "inc/dec" amount to devide the days' interval

In [14]:
df[['INC_AMOUNT_4W', 'DEC_AMOUNT_4W']] = 0.0
df[['INC_DAYS_INTERVAL_4W', 'DEC_DAYS_INTERVAL_4W']] = 0
df[['INC_4W_SPEED', 'DEC_4W_SPEED']] = 0.0

ideal_dwjz_4w_min_and_max = df[['FSRQ', 'IDEAL_DWJZ_4W_MAX', 'IDEAL_DWJZ_4W_MIN']]

def woody_get_index_by_value(df, column, value):
    return df[df[column] == value].index

def make_inc_dec_speed(row, df):
    row_index = str(row['FSRQ'])
    
    ideal_dwjz_4w_max = row['IDEAL_DWJZ_4W_MAX']
    ideal_dwjz_4w_min = row['IDEAL_DWJZ_4W_MIN']
    ideal_dwjz = row['IDEAL_DWJZ']
    
    inc_amount = (ideal_dwjz - ideal_dwjz_4w_min) / ideal_dwjz_4w_min * 100
    dec_amount = (ideal_dwjz - ideal_dwjz_4w_max) / ideal_dwjz_4w_max * 100
    
    df.at[row_index, 'INC_AMOUNT_4W'] = inc_amount
    df.at[row_index, 'DEC_AMOUNT_4W'] = dec_amount
    
    row_loc = ideal_dwjz_4w_min_and_max.index.get_loc(row_index)
    row_4w_max_loc = df.index.get_loc(woody_get_index_by_value(df, 'IDEAL_DWJZ', ideal_dwjz_4w_max)[0])
    row_4w_min_loc = df.index.get_loc(woody_get_index_by_value(df, 'IDEAL_DWJZ', ideal_dwjz_4w_min)[0])
    row_inc_days_interval = abs(row_4w_min_loc - row_loc) + 1
    row_dec_days_interval = abs(row_4w_max_loc - row_loc) + 1
    df.at[row_index, 'INC_DAYS_INTERVAL_4W'] = row_inc_days_interval
    df.at[row_index, 'DEC_DAYS_INTERVAL_4W'] = row_dec_days_interval
    
    df.at[row_index, 'INC_4W_SPEED'] = inc_amount / row_inc_days_interval
    df.at[row_index, 'DEC_4W_SPEED'] = dec_amount / row_dec_days_interval
    
    
df.apply(lambda row: make_inc_dec_speed(row, df), axis = 1)

df.head(50)



,LJJZ,JZZZL,FSRQ,DWJZ,IDEAL_DWJZ,LJJZ_2W_MAX,LJJZ_2W_MIN,LJJZ_4W_MAX,LJJZ_4W_MIN,LJJZ_6W_MAX,...,IDEAL_DWJZ_4W_MAX,IDEAL_DWJZ_4W_MIN,LJJZ_2W_INC,LJJZ_2W_DEC,INC_AMOUNT_4W,DEC_AMOUNT_4W,INC_DAYS_INTERVAL_4W,DEC_DAYS_INTERVAL_4W,INC_4W_SPEED,DEC_4W_SPEED
FSRQ,,,,,,,,,,,,,,,,,,,,,
2021-05-07,2.9776,-1.67,2021-05-07,1.3465,5.109489,3.0619,2.9776,3.0619,2.8972,3.0619,...,5.429156,4.804428,3.0619,2.9776,6.349573,-5.887974,18,5,0.352754,-1.177595
2021-05-06,3.0005,-3.05,2021-05-06,1.3694,5.196267,3.0619,2.9794,3.0619,2.8972,3.0619,...,5.429156,4.804428,3.0619,2.9794,8.155774,-4.289610,17,4,0.479751,-1.072403
2021-04-30,3.0436,-1.16,2021-04-30,1.4125,5.359739,3.0619,2.9642,3.0619,2.8972,3.0619,...,5.429156,4.804428,3.0619,2.9642,11.558302,-1.278608,16,3,0.722394,-0.426203
2021-04-29,3.0602,-0.12,2021-04-29,1.4291,5.422641,3.0619,2.9479,3.0619,2.8972,3.0619,...,5.429156,4.804428,3.0619,2.9479,12.867566,-0.120000,15,2,0.857838,-0.060000
2021-04-28,3.0619,2.51,2021-04-28,1.4308,5.429156,3.0619,2.9163,3.0619,2.8951,3.0619,...,5.429156,4.796729,3.0619,2.9163,13.184554,0.000000,20,1,0.659228,0.000000
2021-04-27,3.0268,1.38,2021-04-27,1.3957,5.296221,3.0278,2.9163,3.0278,2.8951,3.0278,...,5.299917,4.796729,3.0278,2.9163,10.413183,-0.069734,19,3,0.548062,-0.023245
2021-04-26,3.0078,-1.43,2021-04-26,1.3767,5.224128,3.0278,2.9099,3.0278,2.8951,3.0278,...,5.299917,4.796729,3.0278,2.9099,8.910222,-1.430000,18,2,0.495012,-0.715000
2021-04-23,3.0278,1.13,2021-04-23,1.3967,5.299917,3.0278,2.8985,3.0278,2.8899,3.0278,...,5.299917,4.776735,3.0278,2.8985,10.952716,0.000000,20,1,0.547636,0.000000
2021-04-22,3.0122,-0.02,2021-04-22,1.3811,5.240697,3.0125,2.8972,3.0125,2.8456,3.0125,...,5.241746,4.608524,3.0125,2.8972,13.717482,-0.020000,20,2,0.685874,-0.010000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1449 entries, 2021-05-07 to 2015-05-27
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LJJZ                  1449 non-null   float64
 1   JZZZL                 1449 non-null   float64
 2   FSRQ                  1449 non-null   object 
 3   DWJZ                  1449 non-null   float64
 4   IDEAL_DWJZ            1449 non-null   float64
 5   LJJZ_2W_MAX           1449 non-null   float64
 6   LJJZ_2W_MIN           1449 non-null   float64
 7   LJJZ_4W_MAX           1449 non-null   float64
 8   LJJZ_4W_MIN           1449 non-null   float64
 9   LJJZ_6W_MAX           1449 non-null   float64
 10  LJJZ_6W_MIN           1449 non-null   float64
 11  IDEAL_DWJZ_4W_MAX     1449 non-null   float64
 12  IDEAL_DWJZ_4W_MIN     1449 non-null   float64
 13  LJJZ_2W_INC           1449 non-null   float64
 14  LJJZ_2W_DEC           1449 non-null   float64
 15  INC_AMOUNT_